In [1]:
import re
import json
import stanza

import numpy as np
import pandas as pd

from itertools import chain
from ast import literal_eval

from preprocessing_functions import load_txt_as_lst, split_txt, \
run_stanza, flatten_list, segment_series, preprocess_series, write_file

# Lucretius en1893 no section markers

In [2]:
# raw_str_path = "/home/craig.car/repos/chiron/align_texts_project/data/lucretius/lucretius_en1893_str.txt"
raw_str_path = "/home/craig.car/repos/chiron/align_texts_project/data/lucretius/lucretius_en1983_morenewlines.txt"
en1893_raw_lst = load_txt_as_lst(raw_str_path)

In [3]:
en1893_raw_lst[:10]

['REMARKS\n',
 '\n',
 'ON THE\n',
 '\n',
 'LIFE AND POEM OF LUCRETIUS.\n',
 '  OF the life of Lucretius but little information has reached us.  Ad nos vix tenuis famæ perlabitur aura.\n',
 '\n',
 'That he was a Roman by birth, is inferred from the pas  sages in his poem in which he speaks of the Roman world as  his country,1 and of the Roman language as his native tongue.2\n',
 '\n',
 'As to the time of his birth, it is stated by Eusebius in his  Chronicon, that he was born in the second year of the hundred  and seventy-first Olympiad, or ninety-five years before Christ.  At this period, Ennius had been dead about seventy years ;  Cicero was in his twelfth year; twenty-five years were to  elapse before the birth of Virgil, and four before that of Julius  Caesar. His style, indeed, would make him seem older, but  its antiquated character may be partly affected, in imitation,  perhaps, of Ennius, for whom he expresses great veneration.3\n']

In [4]:
en1893_str = " ".join(en1893_raw_lst)

In [5]:
en1893_str[:500]

'REMARKS\n \n ON THE\n \n LIFE AND POEM OF LUCRETIUS.\n   OF the life of Lucretius but little information has reached us.  Ad nos vix tenuis famæ perlabitur aura.\n \n That he was a Roman by birth, is inferred from the pas  sages in his poem in which he speaks of the Roman world as  his country,1 and of the Roman language as his native tongue.2\n \n As to the time of his birth, it is stated by Eusebius in his  Chronicon, that he was born in the second year of the hundred  and seventy-first Olympiad, or ni'

In [6]:
# load stanza model for lang
lang_ = "en"
stanza_model_ = stanza.Pipeline(lang=lang_, processors='tokenize', use_gpu=True)

2023-07-25 16:23:14 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-07-25 16:23:14 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |

2023-07-25 16:23:14 WARNING: GPU requested, but is not available!
2023-07-25 16:23:14 INFO: Using device: cpu
2023-07-25 16:23:14 INFO: Loading: tokenize
2023-07-25 16:23:14 INFO: Done loading processors!


In [7]:
en1983_sents = preprocess_series(en1893_str, "en", stanza_model_)

segmented str into sentences


In [8]:
# without additional newlines around notes and titles, len(en1983_sents) was 13966

In [9]:
len(en1983_sents)

14648

In [10]:
en1983_sents[:10]

['REMARKS',
 'ON THE',
 'LIFE AND POEM OF LUCRETIUS.',
 'OF the life of Lucretius but little information has reached us.',
 'Ad nos vix tenuis famæ perlabitur aura.',
 'That he was a Roman by birth, is inferred from the pas  sages in his poem in which he speaks of the Roman world as  his country,1 and of the Roman language as his native tongue.',
 '2',
 'As to the time of his birth, it is stated by Eusebius in his  Chronicon, that he was born in the second year of the hundred  and seventy-first Olympiad, or ninety-five years before Christ.',
 'At this period, Ennius had been dead about seventy years ;',
 'Cicero was in his twelfth year;']

In [11]:
# path_out = "/home/craig.car/repos/chiron/align_texts_project/data/lucretius/lucretius_en1893_sents_NEW.txt"
# write_file(en1983_sents, path_out)

# Lucretius en1893 with section markers

In [12]:
raw_path_markers = "/home/craig.car/repos/chiron/align_texts_project/data/lucretius/lucretius_en1893_str_withsections.txt"
en1893_raw_lst_markers = load_txt_as_lst(raw_path_markers)

In [13]:
en1893_raw_lst_markers[:10]

['#@$% chapter= #@$%\n',
 '\n',
 '#@$%title#@$%REMARKS\n',
 '\n',
 '#@$%title#@$%ON THE\n',
 '   #@$%title#@$%LIFE AND POEM OF LUCRETIUS.\n',
 '  OF the life of Lucretius but little information has reached us.  Ad nos vix tenuis famæ perlabitur aura.\n',
 '\n',
 'That he was a Roman by birth, is inferred from the pas  sages in his poem in which he speaks of the Roman world as  his country,1 and of the Roman language as his native tongue.2\n',
 '\n']

In [14]:
en1893_str_markers = " ".join(en1893_raw_lst_markers)

In [15]:
en1893_str_markers[:1000]

'#@$% chapter= #@$%\n \n #@$%title#@$%REMARKS\n \n #@$%title#@$%ON THE\n    #@$%title#@$%LIFE AND POEM OF LUCRETIUS.\n   OF the life of Lucretius but little information has reached us.  Ad nos vix tenuis famæ perlabitur aura.\n \n That he was a Roman by birth, is inferred from the pas  sages in his poem in which he speaks of the Roman world as  his country,1 and of the Roman language as his native tongue.2\n \n As to the time of his birth, it is stated by Eusebius in his  Chronicon, that he was born in the second year of the hundred  and seventy-first Olympiad, or ninety-five years before Christ.  At this period, Ennius had been dead about seventy years ;  Cicero was in his twelfth year; twenty-five years were to  elapse before the birth of Virgil, and four before that of Julius  Caesar. His style, indeed, would make him seem older, but  its antiquated character may be partly affected, in imitation,  perhaps, of Ennius, for whom he expresses great veneration.3\n \n Concerning his famil

In [16]:
en1893_str_markers = en1893_str_markers.replace("\n", "")

In [17]:
en1893_str_markers[:1000]

'#@$% chapter= #@$%  #@$%title#@$%REMARKS  #@$%title#@$%ON THE    #@$%title#@$%LIFE AND POEM OF LUCRETIUS.   OF the life of Lucretius but little information has reached us.  Ad nos vix tenuis famæ perlabitur aura.  That he was a Roman by birth, is inferred from the pas  sages in his poem in which he speaks of the Roman world as  his country,1 and of the Roman language as his native tongue.2  As to the time of his birth, it is stated by Eusebius in his  Chronicon, that he was born in the second year of the hundred  and seventy-first Olympiad, or ninety-five years before Christ.  At this period, Ennius had been dead about seventy years ;  Cicero was in his twelfth year; twenty-five years were to  elapse before the birth of Virgil, and four before that of Julius  Caesar. His style, indeed, would make him seem older, but  its antiquated character may be partly affected, in imitation,  perhaps, of Ennius, for whom he expresses great veneration.3  Concerning his family nothing is known. The 

## Split by section (including chapters)
Note: "#@$%" added at beginning and end of section markers when extracting text from xml

In [18]:
en1893_str_markers_split = en1893_str_markers.split("#@$%")

In [19]:
en1893_str_markers_split[:20]

['',
 ' chapter= ',
 '  ',
 'title',
 'REMARKS  ',
 'title',
 'ON THE    ',
 'title',
 'LIFE AND POEM OF LUCRETIUS.   OF the life of Lucretius but little information has reached us.  Ad nos vix tenuis famæ perlabitur aura.  That he was a Roman by birth, is inferred from the pas  sages in his poem in which he speaks of the Roman world as  his country,1 and of the Roman language as his native tongue.2  As to the time of his birth, it is stated by Eusebius in his  Chronicon, that he was born in the second year of the hundred  and seventy-first Olympiad, or ninety-five years before Christ.  At this period, Ennius had been dead about seventy years ;  Cicero was in his twelfth year; twenty-five years were to  elapse before the birth of Virgil, and four before that of Julius  Caesar. His style, indeed, would make him seem older, but  its antiquated character may be partly affected, in imitation,  perhaps, of Ennius, for whom he expresses great veneration.3  Concerning his family nothing is kn

In [20]:
# define patterns
chapter_marker = re.compile(" chapter= ") # 4: commentary, prose translation, metric translation, index
title_start = "title"
title_end = "title_end"
title_commentary = re.compile("ARGUMENT") # startswith
title_book = re.compile("BOOK")
title_index = re.compile("INDEX.")
para_end = re.compile("paragraph end")
note_start = re.compile("note")
note_end = re.compile("note_end")

In [21]:
if re.match(title_start, "title_end"):
    print("yes")

yes


In [22]:
test = "ARGUMENT. asoidau"

In [23]:
test.startswith("ARGUMENT.")

True

In [24]:
len(en1893_str_markers_split)

5765

In [25]:
chapteridx2chaptername = {}
chapteridx2chaptername[0] = "foreword"
chapteridx2chaptername[1] = "prose_translation"
chapteridx2chaptername[2] = "metric_translation"
chapteridx2chaptername[3] = "index"

In [26]:
en1893_str_markers_split[:25]

['',
 ' chapter= ',
 '  ',
 'title',
 'REMARKS  ',
 'title',
 'ON THE    ',
 'title',
 'LIFE AND POEM OF LUCRETIUS.   OF the life of Lucretius but little information has reached us.  Ad nos vix tenuis famæ perlabitur aura.  That he was a Roman by birth, is inferred from the pas  sages in his poem in which he speaks of the Roman world as  his country,1 and of the Roman language as his native tongue.2  As to the time of his birth, it is stated by Eusebius in his  Chronicon, that he was born in the second year of the hundred  and seventy-first Olympiad, or ninety-five years before Christ.  At this period, Ennius had been dead about seventy years ;  Cicero was in his twelfth year; twenty-five years were to  elapse before the birth of Virgil, and four before that of Julius  Caesar. His style, indeed, would make him seem older, but  its antiquated character may be partly affected, in imitation,  perhaps, of Ennius, for whom he expresses great veneration.3  Concerning his family nothing is kn

## remove empty strings

In [27]:
df = pd.DataFrame(en1893_str_markers_split)
df.head()

,0
0,
1,chapter=
2,
3,title
4,REMARKS


In [28]:
df.shape

(5765, 1)

In [29]:
df.isna().sum()

0    0
dtype: int64

In [30]:
df[0].replace("   ", np.nan, inplace=True)
df[0].replace("  ", np.nan, inplace=True)
df[0].replace(" ", np.nan, inplace=True)
df[0].replace("", np.nan, inplace=True)

In [31]:
df.isna().sum()

0    1238
dtype: int64

In [32]:
df.dropna(inplace=True)

In [33]:
df.head()

,0
1,chapter=
3,title
4,REMARKS
5,title
6,ON THE


In [34]:
df.shape

(4527, 1)

In [35]:
en1893_str_markers_split = list(df[0])

In [36]:
en1893_str_markers_split[11]

' 2 I. 31; ii. 259. '

## Reconstruct text without markers

In [235]:
en_reconstructed = []
en_reconstructed_idx = -1

chapter_counter = -1
chap2idx_start = {}

book_counter = 0

idx2section_name = {}

hit_section_marker = False

lst_test = []

lucretius_en1893_textonly = []

for section_idx, section in enumerate(en1893_str_markers_split):
    lst_test.append(hit_section_marker)
    
    if section == "":
        continue
    elif section == " ":
        continue
    elif section == "  ":
        continue
        
    # if section is chapter marker
    elif re.match(chapter_marker, section):
        # increment chapter counter
        chapter_counter += 1
        # reset book counter
        book_counter = 0
        # mark idx of en_reconstructed where chapter starts
        chap2idx_start[chapter_counter] = en_reconstructed_idx
    
    # if section is title marker
    elif re.match(title_start, section):
        hit_section_marker = True
        # then next section is a title, which will be added to en_reconstructed
        # label as title for titles that don't match instances below (index, book, argument)
        label_nextitem = str(chapteridx2chaptername[chapter_counter]) + "book" + str(book_counter) + "title"
        
        # if title (next item in list) starts with "INDEX"
        if en1893_str_markers_split[section_idx+1].startswith("INDEX."):
            # index is in one section only; rename label
            label_nextitem = "index"

        # if next item in list starts with "BOOK"
        elif en1893_str_markers_split[section_idx+1].startswith("BOOK"):
            # new book is starting
            book_counter += 1
            # rename label
            label_nextitem = str(chapteridx2chaptername[chapter_counter]) + "book" + str(book_counter) + "title"
        
        # if next item in list starts with "ARGUMENT"
        elif en1893_str_markers_split[section_idx+1].startswith("ARGUMENT"):
            # label as commentary
            label_nextitem = str(chapteridx2chaptername[chapter_counter]) + "book" + str(book_counter) + "commentary"
        
        continue
    
    # if section marks the start of a note
    # elif re.match(note_start, section):
    elif section == "note":
        if en1893_str_markers_split[section_idx+1] == "note_end":
            hit_section_marker = False
            continue
        else:
            hit_section_marker = True
            # then next section is a footnote, add to en_reconstructed
            label_nextitem = str(chapteridx2chaptername[chapter_counter]) + "book" + str(book_counter) + "note"
    
    elif re.match(note_end, section):
        # do nothing; just there to separate footnote from following text
        hit_section_marker = False
        continue
    
    elif re.match(para_end, section):
        # do nothing; just there to create separations in sections from raw text
        hit_section_marker = False
        continue
    
    else:
        # all markers have been excluded; append to en_reconstructed
        en_reconstructed.append(section)
        en_reconstructed_idx += 1
        
        if hit_section_marker == True:
            idx2section_name[en_reconstructed_idx] = label_nextitem
            hit_section_marker = False
            
        else:
            idx2section_name[en_reconstructed_idx] = str(chapteridx2chaptername[chapter_counter]) + "book" + str(book_counter)
            if chapter_counter > 0:
                lucretius_en1893_textonly.append(section)


In [236]:
# fix errors here, then initialize en1983_sents_tokenized again
for sent_idx, sent in enumerate(en_reconstructed):
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(".”1", ".” 1")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace("tongue.2", "tongue. 2")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace("veneration.3", "veneration. 3")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace("insane.3", "insane. 3")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(".1", ". 1")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace("opinion.3", "opinion. 3")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(".7", ". 7")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(".2", ". 2")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(".4", ". 4")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('language;"', 'language; "')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('Lucretius;"', 'Lucretius; "')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace("?1", "? 1")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(".5", ". 5")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(".6", ". 6")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace("It;.", "It; .")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(":8", ": 8")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(".3", ". 3")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(":2", ": 2")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace("?2", "? 2")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('ἔχειπρός.”Wakefield.', 'ἔχειπρός. ”Wakefield.')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace("Tacitus. 11", "Tacitus.11")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('Æolia.”What', 'Æolia. ”What')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('concerned).—', 'concerned). —')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('.—', '. —')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace("nix.—Are", "nix. —Are")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace("improvida.—Find", "improvida. —Find")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(";", "; ")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace("fleshes.”", "fleshes. ”")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(".]Ver", ".] Ver")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(":so", ": so")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace("flumina. —“", "flumina.—“")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace("armis.–Fortè", "armis. –Fortè")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace("—But", "— But")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace("—Or", "— Or")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace("minds.", "minds. ")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(':"', ': "')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('first.-The', 'first. -The')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(':)', ': )')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('body. ——I', 'body.— —I')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('curtains. —“', 'curtains.—“')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('593. —"', '593.—"')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('borne. 3', 'borne.3')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('words.J', 'words. J')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('compose. 1', 'compose.1')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace("systems.”Whewell's", "systems. ”Whewell's")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('life?3', 'life? 3')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace("? 1  Or,", "?1 Or,")
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('true.]1', 'true.] 1')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('Phil. i. 1. 1.', 'Phil. i. 1.1.')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('up. 1 adhering', 'up.1 adhering')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('6, 6. —With', '6, 6.—With')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('THE YEAR. 3', 'THE YEAR.3')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('elbows?3', 'elbows? 3')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(':—idem', ': —idem')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('moisture.-Motion', 'moisture. -Motion')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('ENTIRE WHOLE. 1', 'ENTIRE WHOLE.1')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('sui.”Aul.Gell.', 'sui. ”Aul. Gell.')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('itself?3', 'itself? 3')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('sun?4', 'sun? 4')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('day :—', 'day : —')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('every nerve. — ', 'every nerve.—')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('fury !— But', 'fury ! —But')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('flame. —', 'flame.—')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('concealed. —', 'concealed.—')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('rose:-', 'rose: -')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('exhaustion:—such', 'exhaustion: —such')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('beneath.-On', 'beneath. -On')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('bring:-', 'bring: -')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('Corporeal:—hence', 'Corporeal: —hence')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(':-can', ': -can')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(':-a', ': -a')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(':-by', ': -by')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('.And', '. And')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('death.-Since', 'death. -Since')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(':-some', ': -some')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('ever!"— But', 'ever!" —But')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('peace. —', 'peace.—')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('aid.-No', 'aid. -No')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('man. —', 'man.—')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('?or', '? or')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(':-it', ': -it')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('ill. —', 'ill.—')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('prevail.-Thus', 'prevail. -Thus')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(':-for', ': -for')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('well-spring:—creed', 'well-spring: —creed')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(':different', ': different')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('an:!', 'an: !')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('v. 2. 5', 'v. 2.5')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('v. 1116;', 'v.1116;')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('1. 57,', '1.57,')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace(':voluntary', ': voluntary')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('ib. ;   grow', 'ib. grow')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('in, v. 30', 'in, v.30')
    en_reconstructed[sent_idx] = en_reconstructed[sent_idx].replace('tones ? 1', 'tones ?1')

# Tokenize en1893 with and without section markers

In [237]:
en1983_sents_tokenized = []
for idx, sent in enumerate(en1983_sents):
    en1983_sents_tokenized.append(sent.split())

In [238]:
en_reconstructed_tokenized = []
for idx, sent in enumerate(en_reconstructed):
    en_reconstructed_tokenized.append(sent.split())

In [239]:
num_tokens_sents = 0
for sent in en1983_sents_tokenized:
    num_tokens_sents += len(sent)

num_tokens_chapts = 0
for sent in en_reconstructed_tokenized:
    num_tokens_chapts += len(sent)

In [240]:
num_tokens_chapts == num_tokens_sents

True

In [241]:
print(num_tokens_chapts)
print(num_tokens_sents)

202634
202634


## Find data errors in tokenized docs

In [242]:
en1983_tokens_from_sents = flatten_list(en1983_sents_tokenized)
en1983_tokens_from_chapts = flatten_list(en_reconstructed_tokenized)

In [243]:
en1983_tokens_from_sents == en1983_tokens_from_chapts

True

In [244]:
en1983_tokens_from_sents[135670:135682]

['others.]',
 'Ver.',
 '1282.',
 'Aliena',
 'rogorum—',
 'extructa.',
 'Thus',
 'abruptly',
 'ends',
 'the',
 'description',
 'of']

In [245]:
en1983_tokens_from_chapts[135670:135682]

['others.]',
 'Ver.',
 '1282.',
 'Aliena',
 'rogorum—',
 'extructa.',
 'Thus',
 'abruptly',
 'ends',
 'the',
 'description',
 'of']

In [246]:
for idx, token in enumerate(en1983_tokens_from_chapts):
    if token != en1983_tokens_from_sents[idx]:
        print(idx)

# Get sent idx to section name dict for en1893

In [247]:
def build_sent_to_section_dict(lst_tokenized_sents, lst_tokenized_chapts,
                               dict_chapter_2_section):
    """
    Build dict of sentence idx to section name
    """
    sent_idx_2_section_name = {}
    token_counter = 0 # per section/chapter
    current_section_idx = 0
    for idx_sent, sent in enumerate(lst_tokenized_sents):
        token_counter += len(sent)
        current_chapter_length = len(lst_tokenized_chapts[current_section_idx])
        if token_counter < current_chapter_length:
            # add sent to dict
            sent_idx_2_section_name[idx_sent] = dict_chapter_2_section[current_section_idx]
        elif token_counter == current_chapter_length:
            # add sent to dict as part of current section
            sent_idx_2_section_name[idx_sent] = dict_chapter_2_section[current_section_idx]
            # reset token counter and current section idx for next sent iteration
            token_counter = 0
            current_section_idx += 1
        else: # token_counter > current_chapter_length, i.e. we've crossed a section boundary 
            # add sent to current section and next section
            sent_idx_2_section_name[idx_sent] = [
                dict_chapter_2_section[current_section_idx], 
                dict_chapter_2_section[current_section_idx+1]]
            # adjust token counter by only including portion of sent in new section
            token_counter = token_counter - current_chapter_length
            # update current section idx for next sent iteration
            current_section_idx += 1
    return sent_idx_2_section_name

In [248]:
en1893_sent2section_name = build_sent_to_section_dict(
    en1983_sents_tokenized, en_reconstructed_tokenized, idx2section_name)


## Write to json

In [222]:
# path_out = "/home/craig.car/repos/chiron/align_texts_project/data/lucretius/en1893_sent2section_dict_jul25.json"
# with open(path_out, 'w') as fp:
#     json.dump(en1893_sent2section_name, fp)

In [52]:
# path_out = "/home/craig.car/repos/chiron/align_texts_project/data/lucretius/en1893_sent2section_dict_NEW.json"
# with open(path_out, 'w') as fp:
#     json.dump(en1893_sent2section_name, fp)

# Get Lucretius text only

In [53]:
prose_sents = ['prose_translationbook1',
               'prose_translationbook2',
               'prose_translationbook3',
               'prose_translationbook4',
               'prose_translationbook5',
               'prose_translationbook6']

metric_sents = ['metric_translationbook1',
                'metric_translationbook2',
                'metric_translationbook3',
                'metric_translationbook4',
                'metric_translationbook5',
                'metric_translationbook6']

In [54]:
# get raw text again and do not remove newlines (as did when building en_reconstructed)
# raw_path_markers = "/home/craig.car/repos/chiron/align_texts_project/data/lucretius/lucretius_en1893_str_withsections.txt"
raw_path_markers = "/home/craig.car/repos/chiron/align_texts_project/data/lucretius/lucretius_en1893_str_withsections_titleend.txt"
en1893_raw_withnewlines = load_txt_as_lst(raw_path_markers)

In [55]:
en1893_raw_withnewlines[:10]

['#@$% chapter= #@$%\n',
 '\n',
 '#@$%title#@$%REMARKS\n',
 '#@$%title_end#@$%\n',
 '#@$%paragraph end#@$%\n',
 '#@$%title#@$%ON THE\n',
 '#@$%title_end#@$%   #@$%title#@$%LIFE AND POEM OF LUCRETIUS.\n',
 '#@$%title_end#@$%  OF the life of Lucretius but little information has reached us.  Ad nos vix tenuis famæ perlabitur aura.\n',
 '#@$%paragraph end#@$%\n',
 'That he was a Roman by birth, is inferred from the pas  sages in his poem in which he speaks of the Roman world as  his country,1 and of the Roman language as his native tongue.2\n']

In [56]:
en1893_markers_newlines = " ".join(en1893_raw_withnewlines)

In [57]:
en1893_markers_newlines[:500]

'#@$% chapter= #@$%\n \n #@$%title#@$%REMARKS\n #@$%title_end#@$%\n #@$%paragraph end#@$%\n #@$%title#@$%ON THE\n #@$%title_end#@$%   #@$%title#@$%LIFE AND POEM OF LUCRETIUS.\n #@$%title_end#@$%  OF the life of Lucretius but little information has reached us.  Ad nos vix tenuis famæ perlabitur aura.\n #@$%paragraph end#@$%\n That he was a Roman by birth, is inferred from the pas  sages in his poem in which he speaks of the Roman world as  his country,1 and of the Roman language as his native tongue.2\n #@$'

In [58]:
en1893_str_markers_newlines_split = en1893_markers_newlines.split("#@$%")

In [59]:
# Lose newlines at end of extracted chunks after splitting
en1893_str_markers_newlines_split[:25]

['',
 ' chapter= ',
 '\n \n ',
 'title',
 'REMARKS\n ',
 'title_end',
 '\n ',
 'paragraph end',
 '\n ',
 'title',
 'ON THE\n ',
 'title_end',
 '   ',
 'title',
 'LIFE AND POEM OF LUCRETIUS.\n ',
 'title_end',
 '  OF the life of Lucretius but little information has reached us.  Ad nos vix tenuis famæ perlabitur aura.\n ',
 'paragraph end',
 '\n That he was a Roman by birth, is inferred from the pas  sages in his poem in which he speaks of the Roman world as  his country,1 and of the Roman language as his native tongue.2\n ',
 'paragraph end',
 '\n As to the time of his birth, it is stated by Eusebius in his  Chronicon, that he was born in the second year of the hundred  and seventy-first Olympiad, or ninety-five years before Christ.  At this period, Ennius had been dead about seventy years ;  Cicero was in his twelfth year; twenty-five years were to  elapse before the birth of Virgil, and four before that of Julius  Caesar. His style, indeed, would make him seem older, but  its antiquat

In [60]:
"title_end" == "title_end"

True

In [385]:
def reconstruct_txt_markers(text_markers_split, chapteridx2chaptername):
    en_reconstructed = []
    en_reconstructed_idx = -1

    chapter_counter = -1
    chap2idx_start = {}

    book_counter = 0

    idx2section_name = {}

    hit_section_marker = False
    hit_index = False
    hit_endprose6 = False
    hit_endprose6commentary = False

    lst_test = []

    lucretius_en1893_textonly = []

    for section_idx, section in enumerate(text_markers_split):
        lst_test.append(hit_section_marker)

        if section == "":
            continue
        elif section == " ":
            continue
        elif section == "  ":
            continue
        elif section == "   ":
            continue
        elif section == "\n ":
            continue
        elif section == "\n \n ":
            continue

        # if section is chapter marker
        elif re.match(chapter_marker, section):
            # increment chapter counter
            chapter_counter += 1
            # reset book counter
            book_counter = 0
            # mark idx of en_reconstructed where chapter starts
            chap2idx_start[chapter_counter] = en_reconstructed_idx

        # if section is title start marker
        elif section == "title":
            hit_section_marker = True
            # then next section is a title, which will be added to en_reconstructed
            # label as title for titles that don't match instances below (index, argument)
            label_nextitem = str(chapteridx2chaptername[chapter_counter]) + "book" + str(book_counter) + "title"

            # if title (next item in list) starts with "INDEX"
            if text_markers_split[section_idx+1].startswith("INDEX."):
                # remaining sections are index
                hit_index = True
                label_nextitem = "index"
                
            # if next item in list starts with "BOOK"
            elif text_markers_split[section_idx+1].startswith("BOOK"):
                # new book is starting
                book_counter += 1
                # # in metric translation, Book title is appended to beginning of first book verse
                # if chapter_counter == 2:
                #     hit_section_marker = False
                    
            # if next item in list starts with "ARGUMENT"
            elif text_markers_split[section_idx+1].startswith("ARGUMENT."):
                # label as commentary
                label_nextitem = str(chapteridx2chaptername[chapter_counter]) + "book" + str(book_counter) + "commentary"

            continue

        # if section marks the start of a note
        # elif re.match(note_start, section):
        elif section == "note":
            if text_markers_split[section_idx+1] == "note_end":
                hit_section_marker = False
                continue
            else:
                hit_section_marker = True
                # then next section is a footnote, add to en_reconstructed
                label_nextitem = str(chapteridx2chaptername[chapter_counter]) + "book" + str(book_counter) + "note"

        elif re.match(note_end, section):
            # do nothing; just there to separate footnote from following text
            hit_section_marker = False
            continue

        elif re.match(para_end, section):
            # do nothing; just there to create separations in sections from raw text
            hit_section_marker = False
            continue
        
        elif section == "title_end":
            # label section following "ARGUMENT" as commentary
            if text_markers_split[section_idx-1].startswith("ARGUMENT."):
                hit_section_marker = True
                label_nextitem = str(chapteridx2chaptername[chapter_counter]) + "book" + str(book_counter) + "commentary"
            continue
        
        else:
            # all markers have been excluded; append to en_reconstructed
            en_reconstructed.append(section)
            en_reconstructed_idx += 1

            if hit_section_marker == True:
                idx2section_name[en_reconstructed_idx] = label_nextitem
                hit_section_marker = False
            
            elif hit_index == True:
                idx2section_name[en_reconstructed_idx] = "index"
            
            elif "Thus abruptly ends the description of the plague," in section:
                idx2section_name[en_reconstructed_idx] = "prose_translationbook6commentary"
                hit_endprose6 = True
            
            elif "If the reader wish to see more accounts of pestilence" in section:
                idx2section_name[en_reconstructed_idx] = "prose_translationbook6commentary"
                hit_endprose6 = False
                
            elif hit_endprose6 == True:
                idx2section_name[en_reconstructed_idx] = "prose_translationbook6commentary"
                # if chapter_counter == 1:
                #     idx2section_name[en_reconstructed_idx] = "prose_translationbook6commentary"
                # elif chapter_counter == 2:
                #     # idx2section_name[en_reconstructed_idx] = "prose_translationbook6commentary"
                #     hit_endprose6 = False
            
            elif "THE END." in section:
                idx2section_name[en_reconstructed_idx] = "metric_translationbook6note"
            
            else:
                idx2section_name[en_reconstructed_idx] = str(chapteridx2chaptername[chapter_counter]) + "book" + str(book_counter)
                if chapter_counter > 0:
                    lucretius_en1893_textonly.append(section)

    return en_reconstructed, idx2section_name, lucretius_en1893_textonly

In [386]:
en_reconstructed_withnewlines, idx2section_name_withnewlines, lucretius_en1893_textonly_withnewlines = reconstruct_txt_markers(
    en1893_str_markers_newlines_split, chapteridx2chaptername)

In [387]:
# apply same changes to en_reconstructed_withnewlines in order to match tokenization
for sent_idx, sent in enumerate(en_reconstructed_withnewlines):
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace(".”1", ".” 1")
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace("tongue.2", "tongue. 2")
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace("veneration.3", "veneration. 3")
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace("insane.3", "insane. 3")
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace(".1", ". 1")
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace("opinion.3", "opinion. 3")
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace(".7", ". 7")
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace(".2", ". 2")
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace(".4", ". 4")
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('language;"', 'language; "')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('Lucretius;"', 'Lucretius; "')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace("?1", "? 1")
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace(".5", ". 5")
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace(".6", ". 6")
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace("It;.", "It; .")
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace(":8", ": 8")
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace(".3", ". 3")
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace(":2", ": 2")
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace("?2", "? 2")
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('ἔχειπρός.”Wakefield.', 'ἔχειπρός. ”Wakefield.')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace("Tacitus. 11", "Tacitus.11")
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('Æolia.”What', 'Æolia. ”What')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('concerned).—', 'concerned). —')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('.—', '. —')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace("nix.—Are", "nix. —Are")
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace("improvida.—Find", "improvida. —Find")
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace(";", "; ")
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace("fleshes.”", "fleshes. ”")
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace(".]Ver", ".] Ver")
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace(":so", ": so")
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace("flumina. —“", "flumina.—“")
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace("armis.–Fortè", "armis. –Fortè")
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace("—But", "— But")
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace("—Or", "— Or")
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace("minds.", "minds. ")
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace(':"', ': "')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('first.-The', 'first. -The')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace(':)', ': )')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('body. ——I', 'body.— —I')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('curtains. —“', 'curtains.—“')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('593. —"', '593.—"')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('borne. 3', 'borne.3')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('words.J', 'words. J')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('compose. 1', 'compose.1')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace("systems.”Whewell's", "systems. ”Whewell's")
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('life?3', 'life? 3')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace("? 1  Or,", "?1 Or,")
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('true.]1', 'true.] 1')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('Phil. i. 1. 1.', 'Phil. i. 1.1.')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('up. 1 adhering', 'up.1 adhering')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('6, 6. —With', '6, 6.—With')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('THE YEAR. 3', 'THE YEAR.3')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('elbows?3', 'elbows? 3')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace(':—idem', ': —idem')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('moisture.-Motion', 'moisture. -Motion')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('ENTIRE WHOLE. 1', 'ENTIRE WHOLE.1')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('sui.”Aul.Gell.', 'sui. ”Aul. Gell.')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('itself?3', 'itself? 3')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('sun?4', 'sun? 4')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('day :—', 'day : —')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('every nerve. — ', 'every nerve.—')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('fury !— But', 'fury ! —But')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('flame. —', 'flame.—')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('concealed. —', 'concealed.—')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('rose:-', 'rose: -')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('exhaustion:—such', 'exhaustion: —such')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('beneath.-On', 'beneath. -On')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('bring:-', 'bring: -')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('Corporeal:—hence', 'Corporeal: —hence')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace(':-can', ': -can')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace(':-a', ': -a')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace(':-by', ': -by')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('.And', '. And')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('death.-Since', 'death. -Since')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace(':-some', ': -some')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('ever!"— But', 'ever!" —But')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('peace. —', 'peace.—')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('aid.-No', 'aid. -No')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('man. —', 'man.—')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('?or', '? or')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace(':-it', ': -it')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('ill. —', 'ill.—')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('prevail.-Thus', 'prevail. -Thus')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace(':-for', ': -for')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('well-spring:—creed', 'well-spring: —creed')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace(':different', ': different')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('an:!', 'an: !')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('v. 2. 5', 'v. 2.5')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('v. 1116;', 'v.1116;')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('1. 57,', '1.57,')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace(':voluntary', ': voluntary')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('ib. ;   grow', 'ib. grow')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('in, v. 30', 'in, v.30')
    en_reconstructed_withnewlines[sent_idx] = en_reconstructed_withnewlines[sent_idx].replace('tones ? 1', 'tones ?1')

In [388]:
len(idx2section_name_withnewlines)

3399

In [389]:
idx2section_name_withnewlines[2009]

'metric_translationbook1'

In [390]:
en1893_textonly_newlines_fromdict = []
idx_counter = 0
idx2section_name_textonly = {}
for key in range(len(idx2section_name_withnewlines)):
    if idx2section_name_withnewlines[key] in prose_sents:
        en1893_textonly_newlines_fromdict.append(en_reconstructed_withnewlines[key])
        idx2section_name_textonly[idx_counter] = idx2section_name_withnewlines[key]
        idx_counter += 1
    elif idx2section_name_withnewlines[key] in metric_sents:
        en1893_textonly_newlines_fromdict.append(en_reconstructed_withnewlines[key])
        idx2section_name_textonly[idx_counter] = idx2section_name_withnewlines[key]
        idx_counter += 1

In [391]:
# check there's a space after colon in "fruit-producing earth; 5 since"
for idx, item in enumerate(en1893_textonly_newlines_fromdict):
    if "O BOUN riFUL" in item:
        print(item)

 O BOUN riFUL Venus,1 mother of the race of Æneas,2 delight  of gods and men, who, beneath the gliding constellations  of heaven,3 fillest with life4 the ship-bearing sea and the  fruit-producing earth; 5 since by thy influence every kind of
 


In [392]:
len(en1893_textonly_newlines_fromdict)

1743

In [393]:
len(en_reconstructed_withnewlines)

3399

## TEST DICT: idx2section_name_textonly (chunk-level from xml)

In [394]:
# Prose Book 1: 0 - 139
for keys in range(0,140):
    if idx2section_name_textonly[keys] != "prose_translationbook1":
        print(keys)

In [395]:
# Prose Book 2: 140 - 276
for keys in range(140, 277):
    if idx2section_name_textonly[keys] != "prose_translationbook2":
        print(keys)

In [396]:
# Prose Book 3: 277 - 415
for keys in range(281, 416):
    if idx2section_name_textonly[keys] != "prose_translationbook3":
        print(keys)

In [397]:
# Prose Book 4: 416 - 576
for keys in range(416, 577):
    if idx2section_name_textonly[keys] != "prose_translationbook4":
        print(keys)

In [398]:
# Prose Book 5: 577 - 764
for keys in range(577, 765):
    if idx2section_name_textonly[keys] != "prose_translationbook5":
        print(keys)

In [399]:
# Prose Book 6: 765 - 967
for keys in range(765, 968):
    if idx2section_name_textonly[keys] != "prose_translationbook6":
        print(keys)

In [400]:
# Metric Book 1: 968 - 1075
for keys in range(968, 1076):
    if idx2section_name_textonly[keys] != "metric_translationbook1":
        print(keys)

In [401]:
# Metric Book 2: 1076 - 1187
for keys in range(1076, 1188):
    if idx2section_name_textonly[keys] != "metric_translationbook2":
        print(keys)

In [402]:
# Metric Book 3: 1188 - 1305
for keys in range(1188, 1306):
    if idx2section_name_textonly[keys] != "metric_translationbook3":
        print(keys)

In [403]:
# Metric Book 4: 1306 - 1415
for keys in range(1306, 1416):
    if idx2section_name_textonly[keys] != "metric_translationbook4":
        print(keys)

In [404]:
# Metric Book 5: 1416 - 1571
for keys in range(1416, 1572):
    if idx2section_name_textonly[keys] != "metric_translationbook5":
        print(keys)

In [405]:
# Metric Book 6: 1572 - 1742 (end)
for keys in range(1572, 1743):
    if idx2section_name_textonly[keys] != "metric_translationbook6":
        print(keys)

In [408]:
en1893_textonly_newlines_fromdict[967]

'\n Dire poverty, too, with sudden impulse, prompted men to  many impious deeds;  for they placed their relatives, with  loud outcries, on the funeral piles raised for others,3 and ap  plied torches to them;  often even quarrelling, with great  bloodshed, rather than the bodies should be left unconsumed.\n '

In [410]:
idx2section_name_textonly[967]

'prose_translationbook6'

## Debug en_reconstructed_withnewlines

In [411]:
for idx, item in enumerate(en_reconstructed_withnewlines):
    if "PARENT of ROME !" in item:
        print(item)
        print(idx)

  PARENT of ROME ! by gods and men beloved,  Benignant VENUS! thou, the sail-clad main  And fruitful earth, as round the seasons roll,  With life who swellest, for by thee all live,  And, living, hail the cheerful light of day : —  Thee, goddess, at thy glad approach, the winds.  The tempests fly : dedalian Earth to thee  Pours forth her sweetest flow'rets: Ocean laughs,  And the blue heavens in cloudless splendour decked.  For, when the Spring first opes her frolic eye,  And genial zephyrs long locked up respire,  Thee, goddess, then, th' aerial birds confess,  To rapture stung through every shivering plume:  Thee, the wild herds ;  hence, o'er the joyous glebe  Bounding at large;  or, with undaunted chest,  Stemming the torrent tides. Through all that lives  So, by thy charms, thy blandishments overpowered,  Springs the warm wish thy footsteps to pursue:  Till through the seas, the mountains, and the floods,  The verdant meads, and woodlands filled with song  Spurred by desire each p

In [412]:
en_reconstructed_withnewlines[2009]

"  PARENT of ROME ! by gods and men beloved,  Benignant VENUS! thou, the sail-clad main  And fruitful earth, as round the seasons roll,  With life who swellest, for by thee all live,  And, living, hail the cheerful light of day : —  Thee, goddess, at thy glad approach, the winds.  The tempests fly : dedalian Earth to thee  Pours forth her sweetest flow'rets: Ocean laughs,  And the blue heavens in cloudless splendour decked.  For, when the Spring first opes her frolic eye,  And genial zephyrs long locked up respire,  Thee, goddess, then, th' aerial birds confess,  To rapture stung through every shivering plume:  Thee, the wild herds ;  hence, o'er the joyous glebe  Bounding at large;  or, with undaunted chest,  Stemming the torrent tides. Through all that lives  So, by thy charms, thy blandishments overpowered,  Springs the warm wish thy footsteps to pursue:  Till through the seas, the mountains, and the floods,  The verdant meads, and woodlands filled with song  Spurred by desire each 

In [413]:
idx2section_name_withnewlines[2009]

'metric_translationbook1'

## Segment text only into sentences

In [80]:
# strip newlines at beginning of each chunk
en1893_textonly_newlines_fromdict = [x.lstrip() for x in en1893_textonly_newlines_fromdict]

In [81]:
en1893_textonly_newlines_fromdict[0]

'O BOUN riFUL Venus,1 mother of the race of Æneas,2 delight  of gods and men, who, beneath the gliding constellations  of heaven,3 fillest with life4 the ship-bearing sea and the  fruit-producing earth; 5 since by thy influence every kind of\n '

In [82]:
en1893_textonly_newlines_fromdict = [x.rstrip() for x in en1893_textonly_newlines_fromdict]

In [83]:
en1893_textonly_newlines_fromdict[0]

'O BOUN riFUL Venus,1 mother of the race of Æneas,2 delight  of gods and men, who, beneath the gliding constellations  of heaven,3 fillest with life4 the ship-bearing sea and the  fruit-producing earth; 5 since by thy influence every kind of'

In [84]:
en1893_str_textonly = " ".join(en1893_textonly_newlines_fromdict)

In [85]:
en1893_sents_textonly = preprocess_series(en1893_str_textonly, "en", stanza_model_)

segmented str into sentences


In [86]:
en1893_sents_textonly[:10]

['O BOUN riFUL Venus,1 mother of the race of Æneas,2 delight  of gods and men, who, beneath the gliding constellations  of heaven,3 fillest with life4 the ship-bearing sea and the  fruit-producing earth;',
 '5 since by thy influence every kind of living creature is conceived, and, springing forth, hails the  light of the sun.',
 'Thee, 0 goddess, thee the winds flee ;',
 'be  fore thee, and thy approach, the clouds of heaven disperse;',
 'for  thee the variegated earth2 puts forth3 her fragrant flowers;',
 'on thee the waters of ocean smile, and the calmed heaven  beams with effulgent4 light.',
 'For, as soon as the vernal face  of day5 is unveiled, and the genial gale of Favonius exerts  its power unconfined, the birds of the air first, 0 goddess,  testify of thee and thy coming, smitten in heart by thy influ  ence.',
 'Next, the wild herds bound over the joyous pastures,  and swim across the rapid streams.',
 'So all kinds of living  creatures, captivated by thy charms and thy allure

In [87]:
en1893_sents_textonly[-10:]

['At length the temples of the gods themselves,  Changed into charnels, and their sacred shrines  Thronged with the dead:',
 'for superstition now,',
 'The power of altars, half their sway had lost,  Whelmed in the pressure of the present woe.',
 'Nor longer now the costly rites prevailed  Of ancient burial, erst punctilious kept:',
 'For all roved restless, with distracted mind,  From scene to scene ;',
 "and worn with grief and toil  Gave to their friends th' interment chance allowed.",
 'And direst exigence impelled them oft,  Headlong, to deeds most impious;',
 "for the pyres  Funereal seized they, reared not by themselves,  And with loud dirge, and wailing wild, o'er these  Placed their own dead;",
 "amid th' unhallowed blaze  With blood contending, rather than resign",
 "The tomb thus gained, or quit th' enkindling corse."]

In [88]:
len(en1893_sents_textonly)

8165

In [89]:
# path_out = "/home/craig.car/repos/chiron/align_texts_project/data/lucretius/lucretius_en1893_sents_textonly.txt"
# write_file(en1893_sents_textonly, path_out)

## TODO: text only experiment
1. tokenize en1893_sents_textonly and tokenize en1893_textonly_newlines_fromdict
2. check they match
3. build dict sent 2 section name for evaluation

# TODO: full edition experiment
1. TEST new dict (see end of notebook)
2. Run through score vec results notebook
3. Check results
4. Update numbers in paper

In [414]:
en_reconstructed_TEST, idx2section_name_TEST, lucretius_en1893_textonly_TEST = reconstruct_txt_markers(
    en1893_str_markers_newlines_split, chapteridx2chaptername)

In [415]:
def reconcile_sents_chapts(en_reconstructed_lst):
    # apply same changes to en_reconstructed_lst in order to match tokenization
    for sent_idx, sent in enumerate(en_reconstructed_lst):
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace(".”1", ".” 1")
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace("tongue.2", "tongue. 2")
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace("veneration.3", "veneration. 3")
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace("insane.3", "insane. 3")
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace(".1", ". 1")
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace("opinion.3", "opinion. 3")
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace(".7", ". 7")
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace(".2", ". 2")
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace(".4", ". 4")
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('language;"', 'language; "')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('Lucretius;"', 'Lucretius; "')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace("?1", "? 1")
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace(".5", ". 5")
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace(".6", ". 6")
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace("It;.", "It; .")
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace(":8", ": 8")
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace(".3", ". 3")
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace(":2", ": 2")
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace("?2", "? 2")
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('ἔχειπρός.”Wakefield.', 'ἔχειπρός. ”Wakefield.')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace("Tacitus. 11", "Tacitus.11")
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('Æolia.”What', 'Æolia. ”What')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('concerned).—', 'concerned). —')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('.—', '. —')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace("nix.—Are", "nix. —Are")
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace("improvida.—Find", "improvida. —Find")
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace(";", "; ")
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace("fleshes.”", "fleshes. ”")
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace(".]Ver", ".] Ver")
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace(":so", ": so")
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace("flumina. —“", "flumina.—“")
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace("armis.–Fortè", "armis. –Fortè")
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace("—But", "— But")
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace("—Or", "— Or")
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace("minds.", "minds. ")
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace(':"', ': "')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('first.-The', 'first. -The')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace(':)', ': )')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('body. ——I', 'body.— —I')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('curtains. —“', 'curtains.—“')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('593. —"', '593.—"')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('borne. 3', 'borne.3')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('words.J', 'words. J')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('compose. 1', 'compose.1')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace("systems.”Whewell's", "systems. ”Whewell's")
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('life?3', 'life? 3')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace("? 1  Or,", "?1 Or,")
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('true.]1', 'true.] 1')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('Phil. i. 1. 1.', 'Phil. i. 1.1.')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('up. 1 adhering', 'up.1 adhering')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('6, 6. —With', '6, 6.—With')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('THE YEAR. 3', 'THE YEAR.3')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('elbows?3', 'elbows? 3')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace(':—idem', ': —idem')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('moisture.-Motion', 'moisture. -Motion')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('ENTIRE WHOLE. 1', 'ENTIRE WHOLE.1')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('sui.”Aul.Gell.', 'sui. ”Aul. Gell.')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('itself?3', 'itself? 3')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('sun?4', 'sun? 4')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('day :—', 'day : —')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('every nerve. — ', 'every nerve.—')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('fury !— But', 'fury ! —But')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('flame. —', 'flame.—')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('concealed. —', 'concealed.—')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('rose:-', 'rose: -')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('exhaustion:—such', 'exhaustion: —such')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('beneath.-On', 'beneath. -On')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('bring:-', 'bring: -')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('Corporeal:—hence', 'Corporeal: —hence')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace(':-can', ': -can')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace(':-a', ': -a')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace(':-by', ': -by')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('.And', '. And')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('death.-Since', 'death. -Since')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace(':-some', ': -some')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('ever!"— But', 'ever!" —But')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('peace. —', 'peace.—')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('aid.-No', 'aid. -No')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('man. —', 'man.—')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('?or', '? or')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace(':-it', ': -it')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('ill. —', 'ill.—')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('prevail.-Thus', 'prevail. -Thus')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace(':-for', ': -for')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('well-spring:—creed', 'well-spring: —creed')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace(':different', ': different')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('an:!', 'an: !')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('v. 2. 5', 'v. 2.5')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('v. 1116;', 'v.1116;')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('1. 57,', '1.57,')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace(':voluntary', ': voluntary')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('ib. ;   grow', 'ib. grow')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('in, v. 30', 'in, v.30')
        en_reconstructed_lst[sent_idx] = en_reconstructed_lst[sent_idx].replace('tones ? 1', 'tones ?1')
    return en_reconstructed_lst

In [416]:
en_reconstructed_TEST_rec = reconcile_sents_chapts(en_reconstructed_TEST)

In [417]:
en1893_sents_tokenized_TEST = []
for idx, sent in enumerate(en1983_sents):
    en1893_sents_tokenized_TEST.append(sent.split())

In [418]:
en_reconstructed_tokenized_TEST = []
for idx, sent in enumerate(en_reconstructed_TEST):
    en_reconstructed_tokenized_TEST.append(sent.split())

In [419]:
num_tokens_sents = 0
for sent in en1893_sents_tokenized_TEST:
    num_tokens_sents += len(sent)

num_tokens_chapts = 0
for sent in en_reconstructed_tokenized_TEST:
    num_tokens_chapts += len(sent)

In [420]:
print(num_tokens_sents)
print(num_tokens_chapts)

202634
202634


In [421]:
en1983_tokens_from_sents_TEST = flatten_list(en1893_sents_tokenized_TEST)
en1983_tokens_from_chapts_TEST = flatten_list(en_reconstructed_tokenized_TEST)

In [422]:
en1983_tokens_from_sents_TEST == en1983_tokens_from_chapts_TEST

True

In [423]:
en1983_tokens_from_chapts_TEST[61:65]

['tongue.', '2', 'As', 'to']

In [424]:
for idx, token in enumerate(en1983_tokens_from_chapts_TEST):
    if token != en1983_tokens_from_sents_TEST[idx]:
        print(idx)

In [425]:
en1893_sent2section_name_TEST = build_sent_to_section_dict(
    en1893_sents_tokenized_TEST, en_reconstructed_tokenized_TEST, idx2section_name_TEST)

In [426]:
for idx, item in enumerate(en1983_sents):
    if "Thus abruptly ends" in item:
        print(item)
        print(idx)

Thus abruptly ends the description of the plague, and the poem  of Lucretius.
8576


In [429]:
en1893_sent2section_name_TEST[8576]

'prose_translationbook6commentary'

In [430]:
en1893_sent2section_name_TEST[8614]

'metric_translationbook1'

In [431]:
en1983_sents[8614]

'PARENT of ROME ! by gods and men beloved,  Benignant VENUS!'

In [432]:
for idx, item in enumerate(en_reconstructed_TEST_rec):
    if "PARENT of ROME ! by" in item:
        print(item)
        print(idx)

  PARENT of ROME ! by gods and men beloved,  Benignant VENUS! thou, the sail-clad main  And fruitful earth, as round the seasons roll,  With life who swellest, for by thee all live,  And, living, hail the cheerful light of day : —  Thee, goddess, at thy glad approach, the winds.  The tempests fly : dedalian Earth to thee  Pours forth her sweetest flow'rets: Ocean laughs,  And the blue heavens in cloudless splendour decked.  For, when the Spring first opes her frolic eye,  And genial zephyrs long locked up respire,  Thee, goddess, then, th' aerial birds confess,  To rapture stung through every shivering plume:  Thee, the wild herds ;  hence, o'er the joyous glebe  Bounding at large;  or, with undaunted chest,  Stemming the torrent tides. Through all that lives  So, by thy charms, thy blandishments overpowered,  Springs the warm wish thy footsteps to pursue:  Till through the seas, the mountains, and the floods,  The verdant meads, and woodlands filled with song  Spurred by desire each p

In [433]:
idx2section_name_TEST[2007]

'metric_translationbook0title'

In [434]:
en_reconstructed_TEST_rec[2007]

'BY JOHN MASON GOOD.\n '

In [435]:
len(en_reconstructed_tokenized_TEST)

3399

In [436]:
len(en_reconstructed_TEST_rec)

3399

In [437]:
def build_sent_to_section_dict(lst_tokenized_sents, lst_tokenized_chapts,
                               dict_chapter_2_section):
    """
    Build dict of sentence idx to section name
    """
    sent_idx_2_section_name = {}
    token_counter = 0 # per section/chapter
    current_section_idx = 0
    for idx_sent, sent in enumerate(lst_tokenized_sents):
        token_counter += len(sent)
        current_chapter_length = len(lst_tokenized_chapts[current_section_idx])
        if token_counter < current_chapter_length:
            # add sent to dict
            sent_idx_2_section_name[idx_sent] = dict_chapter_2_section[current_section_idx]
        elif token_counter == current_chapter_length:
            # add sent to dict as part of current section
            sent_idx_2_section_name[idx_sent] = dict_chapter_2_section[current_section_idx]
            # reset token counter and current section idx for next sent iteration
            token_counter = 0
            current_section_idx += 1
        else: # token_counter > current_chapter_length, i.e. we've crossed a section boundary
            print(idx_sent)
            # add sent to current section and next section
            sent_idx_2_section_name[idx_sent] = [
                dict_chapter_2_section[current_section_idx], 
                dict_chapter_2_section[current_section_idx+1]]
            # adjust token counter by only including portion of sent in new section
            token_counter = token_counter - current_chapter_length
            # update current section idx for next sent iteration
            current_section_idx += 1
    return sent_idx_2_section_name

In [438]:
# path_out = "/home/craig.car/repos/chiron/align_texts_project/data/lucretius/en1893_sent2section_dict_jul25_NEW.json"
# with open(path_out, 'w') as fp:
#     json.dump(en1893_sent2section_name_TEST, fp)

## TEST DICT: sent2section_name full edition

In [ ]:
# Foreword
for keys in range(0,140):
    if en1893_sent2section_name_TEST[keys] != "foreword":
        print(keys)